# required packages

In [1]:
import pandas as pd
import requests
from io import StringIO

from geopy.geocoders import Nominatim
import time

#!pip install folium
import folium

# data upload

In [18]:
github="https://raw.githubusercontent.com/agronomy4future/raw_data_practice/refs/heads/main/US_AGRICULTURE_COLLEGE_LIST.csv"
response=requests.get(github)
df=pd.read_csv(StringIO(response.text))
df.head(10)

,Ranking,Institution,Address
0,2,"University of California, Davis","1 Shields Ave, Davis, CA 95616"
1,4,Cornell University,"410 Thurston Ave, Ithaca, NY 14850"
2,5,Purdue University,"915 Mitch Daniels Blvd, West Lafayette, IN 47907"
3,10,"University of California, Berkeley","University of California, Berkeley, Berkeley, CA"
4,11,Iowa State University,"Iowa State University, Ames, IA 50011"
5,12,University of Wisconsin-Madison,"500 Lincoln Dr, Madison, WI 53706"
6,14,Michigan State University,"426 Auditorium Rd, East Lansing, MI 48824"
7,14,Texas A&M University,"400 Bizzell St, College Station, TX 77840"
8,19,University of Florida,"1478 Union Rd, Gainesville, FL 32611"
9,20,University of Illinois at Urbana-Champaign,"901 W Illinois St, Urbana, IL 61801"


# adding coordinates

In [ ]:
geolocator = Nominatim(user_agent="geoapi")

def get_lat_lon(Address):
    try:
        location = geolocator.geocode(Address)
        return location.latitude, location.longitude
    except:
        return None, None

df[['latitude', 'longitude']] = df['Address'].apply(lambda x: pd.Series(get_lat_lon(x)))

In [20]:
df.head(10)

,Ranking,Institution,Address,latitude,longitude
0,2,"University of California, Davis","1 Shields Ave, Davis, CA 95616",38.540249,-121.749065
1,4,Cornell University,"410 Thurston Ave, Ithaca, NY 14850",42.453952,-76.484291
2,5,Purdue University,"915 Mitch Daniels Blvd, West Lafayette, IN 47907",40.424088,-86.918206
3,10,"University of California, Berkeley","University of California, Berkeley, Berkeley, CA",37.875500,-122.239069
4,11,Iowa State University,"Iowa State University, Ames, IA 50011",42.027961,-93.644737
5,12,University of Wisconsin-Madison,"500 Lincoln Dr, Madison, WI 53706",43.075364,-89.404342
6,14,Michigan State University,"426 Auditorium Rd, East Lansing, MI 48824",42.730091,-84.477668
7,14,Texas A&M University,"400 Bizzell St, College Station, TX 77840",30.625517,-96.339284
8,19,University of Florida,"1478 Union Rd, Gainesville, FL 32611",29.650205,-82.341610
9,20,University of Illinois at Urbana-Champaign,"901 W Illinois St, Urbana, IL 61801",40.108254,-88.220647


# download csv. file to verify all coordinates were generated

In [21]:
df.to_csv('df.csv', index=False)
from google.colab import files
files.download('df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# mount google drive (to save map in my google drive)

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
# Drop rows with missing latitude or longitude values
df_clean = df.dropna(subset=["latitude", "longitude"])

# Define the center of the map
map_center = [df_clean["latitude"].mean(), df_clean["longitude"].mean()]
college_map = folium.Map(location=map_center, zoom_start=11)

# Add red points for each location
for _, row in df_clean.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
    ).add_to(college_map)

# Save and display the interactive map
map_path = "/content/drive/MyDrive/Colab/Python_code/college_map.html"
college_map.save(map_path)
map_path

'/content/drive/MyDrive/Colab/Python_code/college_map.html'

In [38]:
import os
from google.colab import output

# Copy file to /content/ to avoid Google Drive access restrictions
shutil.copy(file_path, "/content/college_map.html")

# Start a web server
os.system("python3 -m http.server 8888 -d /content &")

# Serve the file as an iframe
output.serve_kernel_port_as_iframe(8888, path="college_map.html")

<IPython.core.display.Javascript object>